In [36]:
import os,glob,sys
import subprocess


##### Get access to this data directory from Paul

In [15]:
original_data_dir = '/explore/nobackup/people/pmontesa/outASP/blacksky/BSG-STEREO-117-20221009-125017-40530679-stereo/'
left_img_fn, right_img_fn = sorted(glob.glob(os.path.join(original_data_dir,'*pan.tif')))

In [16]:
left_img_fn

'/explore/nobackup/people/pmontesa/outASP/blacksky/BSG-STEREO-117-20221009-125017-40530679-stereo/BSG-117-20221009-125017-40530679_georeferenced-pan.tif'

## Bookkeeping
Change these hardcorded paths etc here

In [19]:
%cd /panfs/ccds02/nobackup/people/sbhusha1/
!mkdir ostfl
!mkdir ostfl/asp_blacksky_tests
%cd ostfl/asp_blacksky_tests

/panfs/ccds02/nobackup/people/sbhusha1
mkdir: cannot create directory ‘ostfl’: File exists
/panfs/ccds02/nobackup/people/sbhusha1/ostfl/asp_blacksky_tests


In [23]:
asp_dir = '/panfs/ccds02/nobackup/people/sbhusha1/sw/StereoPipeline-3.5.0-alpha-2025-04-17-x86_64-Linux/bin/'

In [24]:
parallel_stereo_cmd = os.path.join(asp_dir,'parallel_stereo')

In [25]:
!$parallel_stereo_cmd --version

NASA Ames Stereo Pipeline 3.5.0-alpha
  Build ID: f7c3cf80
  Build date: 2025-04-17

Built against:
  NASA Vision Workbench 3.5.0-alpha
    Build ID: 3fb7eaf5
  USGS ISIS 8.1.0
  Boost C++ Libraries 1.82.0
  GDAL 3.8.0 | 20231106



In [33]:
def run_bash_command(cmd,verbose=False):
    #written by Scott Henderson
    # move to asp_binder_utils
    """Call a system command through the subprocess python module."""
    print(cmd)
    try:
        if verbose:
            retcode = subprocess.call(cmd,shell=True)
        else:
            retcode = subprocess.call(cmd, stdout=subprocess.DEVNULL,
                stderr=subprocess.STDOUT,shell=True)
        if retcode < 0:
            print("Child was terminated by signal", -retcode, file=sys.stderr)
        else:
            print("Child returned", retcode, file=sys.stderr)
    except OSError as e:
        print("Execution failed:", e, file=sys.stderr)

## Processing setup-1: ASP default block matching

In [34]:
verbose = False #set this to True if want to look at real-time logs

In [37]:
%%time
outprefix = 'asp_block/asp_block'
stereo_algorithm = 'asp_bm'
align_method = 'local_epipolar' #this is key for external plugin algorithms
#keep other settings default for now
stereo_cmd = f"{parallel_stereo_cmd} -t rpc --stereo-algorithm {stereo_algorithm} --alignment-method {align_method}"
stereo_args = f" {left_img_fn} {right_img_fn} {outprefix}"
stereo_execute = stereo_cmd + stereo_args

run_bash_command(stereo_execute,verbose=False)

/panfs/ccds02/nobackup/people/sbhusha1/sw/StereoPipeline-3.5.0-alpha-2025-04-17-x86_64-Linux/bin/parallel_stereo -t rpc --stereo-algorithm asp_bm --alignment-method local_epipolar /explore/nobackup/people/pmontesa/outASP/blacksky/BSG-STEREO-117-20221009-125017-40530679-stereo/BSG-117-20221009-125017-40530679_georeferenced-pan.tif /explore/nobackup/people/pmontesa/outASP/blacksky/BSG-STEREO-117-20221009-125017-40530679-stereo/BSG-117-20221009-125025-40530680_georeferenced-pan.tif asp_block/asp_block
CPU times: user 14.2 ms, sys: 9.14 ms, total: 23.4 ms
Wall time: 4min 13s


Child returned 0


## Processing setup-2: Open-CV Block-matching

In [38]:
%%time
outprefix = 'asp_opencv_block/asp_opencv_block'
stereo_algorithm = '"opencv_bm -block_size 21 -texture_thresh 10 -prefilter_cap 31 -uniqueness_ratio 15 -speckle_size 100 -speckle_range 32 -disp12_diff 1"'
align_method = 'local_epipolar' #this is key for external plugin algorithms
#keep other settings default for now
stereo_cmd = f"{parallel_stereo_cmd} -t rpc --stereo-algorithm {stereo_algorithm} --alignment-method {align_method}"
stereo_args = f" {left_img_fn} {right_img_fn} {outprefix}"
stereo_execute = stereo_cmd + stereo_args

run_bash_command(stereo_execute,verbose=False)

/panfs/ccds02/nobackup/people/sbhusha1/sw/StereoPipeline-3.5.0-alpha-2025-04-17-x86_64-Linux/bin/parallel_stereo -t rpc --stereo-algorithm "opencv_bm -block_size 21 -texture_thresh 10 -prefilter_cap 31 -uniqueness_ratio 15 -speckle_size 100 -speckle_range 32 -disp12_diff 1" --alignment-method local_epipolar /explore/nobackup/people/pmontesa/outASP/blacksky/BSG-STEREO-117-20221009-125017-40530679-stereo/BSG-117-20221009-125017-40530679_georeferenced-pan.tif /explore/nobackup/people/pmontesa/outASP/blacksky/BSG-STEREO-117-20221009-125017-40530679-stereo/BSG-117-20221009-125025-40530680_georeferenced-pan.tif asp_opencv_block/asp_opencv_block
/panfs/ccds02/nobackup/people/sbhusha1/sw/StereoPipeline-3.5.0-alpha-2025-04-17-x86_64-Linux/bin/parallel_stereo -t rpc --stereo-algorithm "opencv_bm -block_size 21 -texture_thresh 10 -prefilter_cap 31 -uniqueness_ratio 15 -speckle_size 100 -speckle_range 32 -disp12_diff 1" --alignment-method local_epipolar /explore/nobackup/people/pmontesa/outASP/bla

Child returned 0


## Processing setup-3: Prepare for Open-CV GPU-based Block-matching using a tiled scheme, using external plugin options

In [42]:
verbose = False

In [43]:
%%time
keep_only = "unchanged" #do not delete the tile directory contents which are made during stereo processing
#the tile directories have *-aligned.tif images, which will need to be passed to gpu correlators as input
outprefix = 'tiled_outputs_for_correlator/asp_local_align'
stop_point = 2 #stop after the locally aligned tiles are prepared after correlation
stereo_algorithm = '"opencv_bm -block_size 21 -texture_thresh 10 -prefilter_cap 31 -uniqueness_ratio 15 -speckle_size 100 -speckle_range 32 -disp12_diff 1"'
align_method = 'local_epipolar' #this is key for external plugin algorithms
#keep other settings default for now
stereo_cmd = f"{parallel_stereo_cmd} -t rpc --stereo-algorithm {stereo_algorithm} --alignment-method {align_method} --stop-point {exit_point} --keep-only {keep_only}"
stereo_args = f" {left_img_fn} {right_img_fn} {outprefix}"
stereo_execute = stereo_cmd + stereo_args

run_bash_command(stereo_execute,verbose=verbose)


/panfs/ccds02/nobackup/people/sbhusha1/sw/StereoPipeline-3.5.0-alpha-2025-04-17-x86_64-Linux/bin/parallel_stereo -t rpc --stereo-algorithm "opencv_bm -block_size 21 -texture_thresh 10 -prefilter_cap 31 -uniqueness_ratio 15 -speckle_size 100 -speckle_range 32 -disp12_diff 1" --alignment-method local_epipolar --stop-point 2 --keep-only unchanged /explore/nobackup/people/pmontesa/outASP/blacksky/BSG-STEREO-117-20221009-125017-40530679-stereo/BSG-117-20221009-125017-40530679_georeferenced-pan.tif /explore/nobackup/people/pmontesa/outASP/blacksky/BSG-STEREO-117-20221009-125017-40530679-stereo/BSG-117-20221009-125025-40530680_georeferenced-pan.tif tiled_outputs_for_correlator/asp_local_align
CPU times: user 3.97 ms, sys: 6.74 ms, total: 10.7 ms
Wall time: 1min 23s


Child returned 0


In [47]:
#note the several tile directories which are outputted from this run
#proceed to next cell
!ls tiled_outputs_for_correlator

asp_local_align-0_0_512_512
asp_local_align-0_1024_512_512
asp_local_align-0_1536_512_512
asp_local_align-0_2048_512_512
asp_local_align-0_2560_512_512
asp_local_align-0_3072_512_512
asp_local_align-0_3584_512_512
asp_local_align-0_4096_512_512
asp_local_align-0_4608_512_512
asp_local_align-0_5120_512_512
asp_local_align-0_512_512_512
asp_local_align-0_5632_512_175
asp_local_align-1024_0_512_512
asp_local_align-1024_1024_512_512
asp_local_align-1024_1536_512_512
asp_local_align-1024_2048_512_512
asp_local_align-1024_2560_512_512
asp_local_align-1024_3072_512_512
asp_local_align-1024_3584_512_512
asp_local_align-1024_4096_512_512
asp_local_align-1024_4608_512_512
asp_local_align-1024_5120_512_512
asp_local_align-1024_512_512_512
asp_local_align-1024_5632_512_175
asp_local_align-1536_0_512_512
asp_local_align-1536_1024_512_512
asp_local_align-1536_1536_512_512
asp_local_align-1536_2048_512_512
asp_local_align-1536_2560_512_512
asp_local_align-1536_3072_512_512
asp_local_align-1536_3584_5

In [49]:
#any of these folders have two tiles which are locally aligned
#see sufix "-left-aligned-tile.tif" and "-right-aligned-tile.tif"
#these tiles are ready to be input to external correlators!
#Example below
!ls tiled_outputs_for_correlator/asp_local_align-512_4096_512_512

512_4096_512_512-aligned-disparity.tif
512_4096_512_512-align-L.exr
512_4096_512_512-align-R.exr
512_4096_512_512-BSG-117-20221009-125017-40530679_georeferenced-pan__BSG-117-20221009-125025-40530680_georeferenced-pan.match
512_4096_512_512-BSG-117-20221009-125017-40530679_georeferenced-pan-stats.tif
512_4096_512_512-BSG-117-20221009-125017-40530679_georeferenced-pan.vwip
512_4096_512_512-BSG-117-20221009-125025-40530680_georeferenced-pan-stats.tif
512_4096_512_512-dirList.txt
512_4096_512_512-Dnosym.tif
512_4096_512_512-D_sub.tif
512_4096_512_512-D.tif
512_4096_512_512-info.txt
512_4096_512_512-left-aligned-tile.tif
512_4096_512_512-lMask_sub.tif
512_4096_512_512-lMask.tif
512_4096_512_512-log-stereo_corr-07-02-1426-1584187.txt
512_4096_512_512-log-stereo_corr-07-02-1428-1595464.txt
512_4096_512_512-lStats.tif
512_4096_512_512-L_sub.tif
512_4096_512_512-L.tif
512_4096_512_512-PC_sub.tif
512_4096_512_512-right-aligned-tile.tif
512_4096_512_512-rMask_sub.tif
512_4096_512_512-rMask.tif
51

## Processing setup-4: Prepare for Open-CV GPU-based Block-matching to produce a single very large tile for initial testing only, using external plugin options

In [65]:
%%time
verbose = False
keep_only = "unchanged" #do not delete the tile directory contents which are made during stereo processing
#the tile directories have *-aligned.tif images, which will need to be passed to gpu correlators as input
outprefix = 'large_tile_output_for_correlator/asp_local_align'
stop_point = 2 #stop after the locally aligned tiles are prepared after correlation
stereo_algorithm = '"opencv_bm -block_size 21 -texture_thresh 10 -prefilter_cap 31 -uniqueness_ratio 15 -speckle_size 100 -speckle_range 32 -disp12_diff 1"'
align_method = 'local_epipolar' #this is key for external plugin algorithms
#keep other settings default for now
tile_size = 8000 # a number bigger than blacksky image extent of 6600, 4400 px
stereo_cmd = f"{parallel_stereo_cmd} --job-size-w {tile_size} --job-size-h {tile_size} -t rpc --stereo-algorithm {stereo_algorithm} --alignment-method {align_method} --stop-point {exit_point} --keep-only {keep_only}"
stereo_args = f" {left_img_fn} {right_img_fn} {outprefix}"
stereo_execute = stereo_cmd + stereo_args

run_bash_command(stereo_execute,verbose=verbose)

/panfs/ccds02/nobackup/people/sbhusha1/sw/StereoPipeline-3.5.0-alpha-2025-04-17-x86_64-Linux/bin/parallel_stereo --job-size-w 8000 --job-size-h 8000 -t rpc --stereo-algorithm "opencv_bm -block_size 21 -texture_thresh 10 -prefilter_cap 31 -uniqueness_ratio 15 -speckle_size 100 -speckle_range 32 -disp12_diff 1" --alignment-method local_epipolar --stop-point 2 --keep-only unchanged /explore/nobackup/people/pmontesa/outASP/blacksky/BSG-STEREO-117-20221009-125017-40530679-stereo/BSG-117-20221009-125017-40530679_georeferenced-pan.tif /explore/nobackup/people/pmontesa/outASP/blacksky/BSG-STEREO-117-20221009-125017-40530679-stereo/BSG-117-20221009-125025-40530680_georeferenced-pan.tif large_tile_output_for_correlator/asp_local_align
CPU times: user 2.02 ms, sys: 7.07 ms, total: 9.09 ms
Wall time: 51.6 s


Child returned 0


In [69]:
## this run only produces one large aligned left and right image
## hopefully this will be easier to work with in initial testing 😊
!ls large_tile_output_for_correlator/asp_local_align-0_0_3193_5807/*aligned-tile.tif -lha

-rw------- 1 sbhusha1 k3004 72M Jul  2 14:44 large_tile_output_for_correlator/asp_local_align-0_0_3193_5807/0_0_3193_5807-left-aligned-tile.tif
-rw------- 1 sbhusha1 k3004 69M Jul  2 14:44 large_tile_output_for_correlator/asp_local_align-0_0_3193_5807/0_0_3193_5807-right-aligned-tile.tif


## What we want
* Eventually, the external GPU-correlator should be able to accept the left and right aligned images as input, and output a disparity map. Once we get there, we can work with Oleg to make decide next steps 😊
* e.g., `external_gpu_corr left_align.tif right_align.tif disparit_result.tif`
* The ouput disparity should have three bands, first band the horizontal disparity, second band the vertical disparity, and the third band as "good-pixel-map", so kind of a boolean array with 1 to where we have valid left and right disparity, and 0 for where the disparity is missing. <we can get rid of this requirement, need to talk with Oleg, and refine from there.>